# User Access Token

## User Acces Token

In [0]:
%sh pip install pyotp



In [0]:
# from azure.keyvault import KeyVaultClient
import requests
import pyotp

In [0]:
## Authenticate to Keyvaults
key_Vault_name = "ia-aat"


In [0]:
grant_type = "password"
client_id = dbutils.secrets.get(key_Vault_name, "idam-client-id")
client_secret = dbutils.secrets.get(key_Vault_name, "idam-secret")
username = dbutils.secrets.get(key_Vault_name, "system-username")
password = dbutils.secrets.get(key_Vault_name, "system-password")
scope = "openid profile roles"

In [0]:
# base_url = "https://idam-api.aat.platform.hmcts.net/o/token"
idam_host = "https://idam-web-public.aat.platform.hmcts.net"
base_url = f"{idam_host}/o/token"

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

In [0]:
# import json
# def generate_user_token(env):
#     #----# determine the instance and Keyvault to use #-----#
#     if env == "sbox":
#         idam_host = "https://idam-web-public.aat.platform.hmcts.net"
#         key_Vault_name = "ia-aat"
#     if env == "stg":
#         idam_host = "https://idam-api.aat.platform.hmcts.net"
#         key_Vault_name = "ia-aat"
#     if env == "prod":
#         idam_host = "" ### Set PROD URL
#         key_Vault_name = "" # SET PROD Key vault

#     base_url = f"{idam_host}/o/token"

#     headers = {
#         "Content-Type": "application/x-www-form-urlencoded"
#     }
#     #-----# Get variables from keyvault #-----#
#     grant_type = "password"
#     client_id = dbutils.secrets.get(key_Vault_name, "idam-client-id")
#     client_secret = dbutils.secrets.get(key_Vault_name, "idam-secret")
#     username = dbutils.secrets.get(key_Vault_name, "system-username")
#     password = dbutils.secrets.get(key_Vault_name, "system-password")
#     scope = "openid profile roles"

#     #-----# Create body of the message #-----#
#     data = {
#         "grant_type": "password",
#         "client_id": client_id,
#         "client_secret": client_secret,
#         "username": username,
#         "password": password,
#         "scope": "openid profile roles"
#     }
#     #-----# POST Request to get user access token #-----#
#     try:

#         idam_response = requests.post(base_url, headers=headers, data=data)
#         # Check and handle response
#         if idam_response.status_code == 200:
#             access_token = idam_response.json()["access_token"]
#             print(idam_response.text)
#             print("Succesfully acquired: User Access Token")
#             return access_token, idam_response.json()["expires_in"]
#         else:
#             print("Failed to get token:", idam_response.status_code)
#             print(idam_response.text)
#     except Exception as e:
#         print("Error:", e)
#         return None

# idam_token,expires_in = generate_user_token("sbox")



In [0]:
from datetime import datetime, timezone, timedelta
import threading
import requests

class IDAMTokenManager:
  def __init__(self,env:str,skew:int=28792):
    self.env = env
    self._token = None
    self._expiration_time = None
    self._uid = None
    self.skew = timedelta(seconds=int(skew))
    self._lock = threading.RLock()

     # ----- Environment config (host + key vault) -----
    if env == "sbox":
        self.idam_host = "https://idam-web-public.aat.platform.hmcts.net"
        key_vault = "ia-aat"
    elif env == "stg":
        self.idam_host = "https://idam-api.aat.platform.hmcts.net"
        key_vault = "ia-aat"
    elif env == "prod":
        # TODO: fill in production host and key vault names.
        self.idam_host = ""
        key_vault = ""
    else:
        # Fail fast if an unknown env is passed.
        raise ValueError(f"Unknown env: {env}")

    # ----- Secrets (fetched once per manager instance) -----
    # Databricks utility to retrieve secrets from Key Vault. ## need to change this to use kv client
    self.client_id = dbutils.secrets.get(key_vault, "idam-client-id")
    self.client_secret = dbutils.secrets.get(key_vault, "idam-secret")
    self.username = dbutils.secrets.get(key_vault, "system-username")
    self.password = dbutils.secrets.get(key_vault, "system-password")

    # OAuth2 token endpoint (password grant).
    self.token_url = f"{self.idam_host}/o/token"
    self.uid_url = f"{self.idam_host}/o/userinfo"
    # Requested scopes.
    self.scope = "openid profile roles"




  def _fetch_token(self):
    data = {
            "grant_type": "password",
            "client_id": self.client_id,
            "client_secret": self.client_secret,
            "username": self.username,
            "password": self.password,
            "scope": self.scope,
        }
    
    headers = {"Content-Type":"application/x-www-form-urlencoded"}

    idam_response = requests.post(self.token_url , headers=headers, data=data)

    if idam_response.status_code != 200:
      raise RuntimeError(f"Token request failed: {idam_response.status_code} {idam_response.text}")


    payload = idam_response.json()

    idam_token = payload.get("access_token")
    expires_in = payload.get("expires_in")
    if not idam_token or not expires_in:
      raise RuntimeError(f"Invalid token response: {payload}")

    now = datetime.now(timezone.utc)

    expiration_time = now + timedelta(seconds=int(expires_in))

    uid = self._fetch_uid(idam_token)

    return idam_token,expiration_time,uid
  

  def _needs_refresh(self):

    if not self._token or not self._expiration_time:
      return True

    return datetime.now(timezone.utc) >= (self._expiration_time - self.skew)


  def get_token(self):
    if not self._needs_refresh():
      return self._token,self._uid
# use locking to only allow one block to refresh and other blocks will have to wait
    with self._lock:
      if self._needs_refresh():
        self._token, self._expiration_time, self._uid = self._fetch_token()
      return self._token, self._uid
    
  def _fetch_uid(self,idam_token):
    uid_headers = {"Authorization": f"Bearer {idam_token}"}
  ## make the get request to get the uid
    try:
        idam_response = requests.get(self.uid_url, headers=uid_headers)
    except Exception as e:        
        print(f"UID request failed: {e}")
  ## safley convert to json
    try:
      payload = idam_response.json()
    except ValueError:
      raise RuntimeError(f"UID endpoint did not return valid JSON: {idam_response.text}")
  ## get the uid from the json
    uid = payload.get("uid")
    if not uid:
        raise RuntimeError(f"UID missing in response: {payload}")
    return uid


    
  def invalidate(self):
    with self._lock:
      self._token = None
      self._expiration_time = None
      self._uid = None

idam_token_mgr = IDAMTokenManager(env="sbox")





In [0]:
idam_token_mgr.get_token()

In [0]:
!pip install azure-keyvault azure-identity

In [0]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

credential = DefaultAzureCredential()
kv_client = SecretClient(vault_url="https://ia-aat.vault.azure.net/", credential=credential)



In [0]:
from datetime import datetime, timezone, timedelta
import threading
import requests
from azure.identity import DefaultAzureCredential
# from azure.keyvault.secrets import SecretClient

class IDAMTokenManager:
  def __init__(self,env:str,skew:int=28792):
    self.env = env
    self._token = None
    self._expiration_time = None
    self._uid = None
    self.skew = timedelta(seconds=int(skew))
    self._lock = threading.RLock()

     # ----- Environment config (host + key vault) -----
    if env == "sbox":
        self.idam_host = "https://idam-web-public.aat.platform.hmcts.net"
        key_vault = "ia-aat"
    elif env == "stg":
        self.idam_host = "https://idam-api.aat.platform.hmcts.net"
        key_vault = "ia-aat"
    elif env == "prod":
        # TODO: fill in production host and key vault names.
        self.idam_host = ""
        key_vault = ""
    else:
        # Fail fast if an unknown env is passed.
        raise ValueError(f"Unknown env: {env}")

    # ----- Secrets (fetched once per manager instance) -----
    # Databricks utility to retrieve secrets from Key Vault. ## need to change this to use kv client

    # credentials = DefaultAzureCredential()
    # kv_client = SecretClient(vault_url=f"https://{key_vault}.vault.azure.net/", credential=credential)
    # self.client_id = kv_client.get_secret("idam-client-id").value
    # self.client_secret = kv_client.get_secret("idam-secret").value
    # self.username = kv_client.get_secret("system-username").value
    # self.password = kv_client.get_secret("system-password").value
    self.client_id = dbutils.secrets.get(key_vault, "idam-client-id")
    self.client_secret = dbutils.secrets.get(key_vault, "idam-secret")
    self.username = dbutils.secrets.get(key_vault, "system-username")
    self.password = dbutils.secrets.get(key_vault, "system-password")

    # OAuth2 token endpoint (password grant).
    self.token_url = f"{self.idam_host}/o/token"
    self.uid_url = f"{self.idam_host}/o/userinfo"
    # Requested scopes.
    self.scope = "openid profile roles"




  def _fetch_token(self):
    data = {
            "grant_type": "password",
            "client_id": self.client_id,
            "client_secret": self.client_secret,
            "username": self.username,
            "password": self.password,
            "scope": self.scope,
        }
    
    headers = {"Content-Type":"application/x-www-form-urlencoded"}

    idam_response = requests.post(self.token_url , headers=headers, data=data)

    if idam_response.status_code != 200:
      raise RuntimeError(f"Token request failed: {idam_response.status_code} {idam_response.text}")


    payload = idam_response.json()

    idam_token = payload.get("access_token")
    expires_in = payload.get("expires_in")
    if not idam_token or not expires_in:
      raise RuntimeError(f"Invalid token response: {payload}")

    now = datetime.now(timezone.utc)

    expiration_time = now + timedelta(seconds=int(expires_in))

    uid = self._fetch_uid(idam_token)

    return idam_token,expiration_time,uid
  

  def _needs_refresh(self):

    if not self._token or not self._expiration_time:
      return True

    return datetime.now(timezone.utc) >= (self._expiration_time - self.skew)


  def get_token(self):
    if not self._needs_refresh():
      return self._token,self._uid
# use locking to only allow one block to refresh and other blocks will have to wait
    with self._lock:
      if self._needs_refresh():
        self._token, self._expiration_time, self._uid = self._fetch_token()
      return self._token, self._uid
    
  def _fetch_uid(self,idam_token):
    uid_headers = {"Authorization": f"Bearer {idam_token}"}
  ## make the get request to get the uid
    try:
        idam_response = requests.get(self.uid_url, headers=uid_headers)
    except Exception as e:        
        print(f"UID request failed: {e}")
  ## safley convert to json
    try:
      payload = idam_response.json()
    except ValueError:
      raise RuntimeError(f"UID endpoint did not return valid JSON: {idam_response.text}")
  ## get the uid from the json
    uid = payload.get("uid")
    if not uid:
        raise RuntimeError(f"UID missing in response: {payload}")
    return uid


    
  def invalidate(self):
    with self._lock:
      self._token = None
      self._expiration_time = None
      self._uid = None

idam_token_mgr = IDAMTokenManager(env="sbox")





In [0]:
idam_token_mgr = IDAMTokenManager(env="sbox")

In [0]:
idam_token,uid = idam_token_mgr.get_token()
print(idam_token)
print(uid)

## Service to Service Token

In [0]:
# %sh
# nslookup rpe-service-auth-provider-aat.service.core-compute-aat.internal


In [0]:
import pyotp
class S2S_Manager():
    def __init__(self,env:str,skew:int=21):
        self.env = env
        self._s2s_token = None
        self.expire_time = None
        self._lock = threading.RLock()
        self._skew = skew

     # ----- Environment config (host + key vault) -----
        if self.env =="sbox":
            self.s2s_host = "rpe-service-auth-provider-aat.service.core-compute-aat.internal"
            self.s2s_ip = "http://10.10.143.250/"
            key_vault = "ia-aat"
        elif self.env =="stg":
            self.s2s_host = None
            self.s2s_ip = None
            key_vault = None
        elif self.env =="prod":
            self.s2s_host = None
            self.s2s_ip = None
            key_vault = None

    # ----- Secrets (fetched once per manager instance) -----
    # Databricks utility to retrieve secrets from Key Vault. ## need to change this to use kv client

        # credentials = DefaultAzureCredential()
        # kv_client = SecretClient(vault_url=f"https://{key_vault}.vault.azure.net/", credential=credential)
        # self.s2s_client_id = kv_client.get_secret("s2s-client-id").value
        self._s2s_secret = dbutils.secrets.get(key_vault, "s2s-secret")
        self.url = f"{self.s2s_ip}lease"
        self.s2s_microservice = "iac"


    def _fetch_s2s_token(self):
        otp = pyotp.TOTP(self._s2s_secret).now()
        # create payload
        s2s_payload = {
            "microservice": self.s2s_microservice,
            "oneTimePassword": otp
        }    
        # Attempt to make request to s2s endpoint
        try:
            now = datetime.now(timezone.utc)
            s2s_response = requests.post(
            self.url,
            json=s2s_payload,
            headers={
                "Host": self.s2s_host,
                "Content-Type": "application/json"
            }
    )
        except Exception as e:
            raise EOFError(f"Error reuesting service to service token: {e}")
        ## Ensure you get a 200 response else raise an error
        if s2s_response.status_code != 200:
            raise RuntimeError(f"Error requesting service to service token: {s2s_response.text}")

        ## Extract token from response
        try:
            payload = s2s_response.text.strip()
            # s2s_token = payload.get("token")
        except Exception as e:
            raise RuntimeError(f"Error extracting service to service token: {e}")
        
        self.expire_time = now + timedelta(seconds=int(self._skew))
        self._s2s_token = payload

        return payload


    def get_token(self):
        if self.expire_time and datetime.now(timezone.utc) < self.expire_time:
            return self._s2s_token
        with self._lock:
            self._s2s_token = self._fetch_s2s_token()
        return self._s2s_token



s2s_manager = S2S_Manager("sbox",21)


    

In [0]:
s2s_token = s2s_manager.get_token()
print(s2s_token)

# CCD API

## Start Case Creation

In [0]:
def start_case_creation(ccd_base_url,uid,jid,ctid,etid,idam_token,s2s_token):


    start_case_endpoint = f"/caseworkers/{uid}/jurisdictions/{jid}/case-types/{ctid}/event-triggers/{etid}/token"

    start_case_creation_url = f"{ccd_base_url}{start_case_endpoint}"

    headers = {
    "Authorization": f"Bearer {idam_token}",        # IDAM user JWT
    "ServiceAuthorization": f"Bearer {s2s_token}",  # service-to-service JWT
    "Accept": "application/json"
    }
    try:
        response = requests.get(start_case_creation_url,headers=headers)
        return response
    except Exception as e:
        print(f"❌ Network error while calling {start_case_creation_url}: {e}")
        return None






In [0]:
jid = "IA"
ctid = "Asylum"
etid = "ariaCreateCase"
env = "sbox"
caseNo = "1234567890123456"

if env == "sbox":
    ccd_base_url = f"https://ccd-data-store-api-ia-case-api-pr-{2750}.preview.platform.hmcts.net"
elif env == "stg":
    ccd_base_url = None
elif env == "prod":
    ccd_base_url = None
else:
    raise ValueError("Invalid environment")


start_response = start_case_creation(ccd_base_url,uid,jid,ctid,etid,idam_token,s2s_token)

if start_response.status_code == 200 :
    event_token = start_response.json()["token"]
else:
    result = {
        "caseNo": caseNo,
        "status": start_response.status_code,
        "message": "Case creation failed",
        "time": datetime.now(timezone.utc).isoformat()
        }
    
result

In [0]:
def validate_case(event_token, payload_data,jid,ctid,idam_token,uid,s2s_token):

    validate_case_endpoint = f"/caseworkers/{uid}/jurisdictions/{jid}/case-types/{ctid}/validate"

    validate_case_url = ccd_base_url + validate_case_endpoint

    headers = {
    "Authorization": f"Bearer {idam_token}",        # IDAM user JWT
    "ServiceAuthorization": f"Bearer {s2s_token}",  # service-to-service JWT
    "Accept": "application/json"
    }


    json_data = json.dumps({
    "data": payload_data,
    "event": {"id":"ariaCreateCase"},
    "event_token": event_token, 
    "ignore_warning": True
    })

    try:
        response = requests.post(validate_case_url,headers=headers,body=json_data)
        return response
    except Exception as e:
        print(f"❌ Network error while calling {validate_case_url}: {e}")
        return None



In [0]:
def submit_case(event_token, payload_data,jid,ctid,idam_token,uid,s2s_token):

    submit_case_endpoint = f"/caseworkers/{uid}/jurisdictions/{jid}/case-types/{ctid}/cases" 

    headers = {
    "Authorization": f"Bearer {idam_token}",        # IDAM user JWT
    "ServiceAuthorization": f"Bearer {s2s_token}",  # service-to-service JWT
    "Accept": "application/json"
    }

    json_data = json.dumps({
    "data": payload_data,
    "event": {"id":"ariaCreateCase"},
    "event_token": event_token, 
    "ignore_warning": True
    })

    submit_case_url = ccd_base_url + submit_case_endpoint 
    try:
        response = requests.post(submit_case_url,headers=headers,body=json_data)
        return response
    except Exception as e:
        print(f"❌ Network error while calling {submit_case_url}: {e}")
        return None





In [0]:
from azure.eventhub import EventHubProducerClient
producer = EventHubProducerClient.from_connection_string(conn_str=connection_str, eventhub_name=eventhub_name)
def submit_json_to_eventhub(producer, result):
    
    try:
        event_data_batch = producer.create_batch()
        event_data_batch.add(EventData(json.dumps(result)))
        producer.send_batch(event_data_batch)
    finally:
        producer.close()

In [0]:
idam_token_mgr = IDAMTokenManager(env="sbox")
idam_token,uid = idam_token_mgr.get_token()
s2s_manager = S2S_Manager("sbox",21)
s2s_token = s2s_manager.get_token()

In [0]:
### caseNo = event.key, payload_data = event.value

def process_case(env,caseNo,payload_data,idam_token,uid,s2s_token,PR_NUMBER=2750):

    jid = "IA"
    ctid = "Asylum"
    etid = "ariaCreateCase"

    urls = {
        "sbox":f"https://ccd-data-store-api-ia-case-api-pr-{PR_NUMBER}.preview.platform.hmcts.net",
        "stg":None,
        "prod":None
    }

    try:
        ccd_base_url = urls[env]
    except KeyError:
        raise ValueError("Invalid environment")

    # if env == "sbox":
    #     ccd_base_url = f"https://ccd-data-store-api-ia-case-api-pr-{PR_NUMBER}.preview.platform.hmcts.net"
    # elif env == "stg":
    #     ccd_base_url = None
    # elif env == "prod":
    #     ccd_base_url = None
    # else:
    #     raise ValueError("Invalid environment")

    ## start case creation

    start_response = start_case_creation(ccd_base_url,uid,jid,ctid,etid,idam_token,s2s_token)

    if start_response is None or start_response.status_code != 200 :

        status_code = start_response.status_code if start_response else "N/A"
        text = start_response.text if start_response else "No response from API"

        print(f"Case creation failed: {status_code} - {text}")

        result = {
        "runID": "get run id from event body",
        "caseNo": caseNo,
        "state": "Add state from event body",
        "status": "ERROR",
        "error": f"Case creation failed: {status_code} - {text}",
        "end_date_time": datetime.now(timezone.utc).isoformat()
        }
        return result
    else:

        event_token = start_response.json()["token"]

    # validate case

    validate_case_response = validate_case(event_token, payload_data)

    if validate_case_response is None or validate_case_response.status_code != 201:
        

        status_code = validate_case_response.status_code if validate_case_response else "N/A"
        text = validate_case_response.text if validate_case_response else "No response from API"

        print(f"Case validation failed: {status_code} - {text}")

        result = {
            "runID": "get run id from event body",
            "caseNo": caseNo,
            "state": "Add state from event body",
            "status": "EEROR", ### change this to the validate response code
            "error": f"Case validation failed: {status_code} - {text}",
            "end_date_time": datetime.now(timezone.utc).isoformat()
        }
        return result

    else:
        print(f"Validation passed for case {caseNo}")

    ## submit case
    submit_case_response = submit_case(event_token, payload_data)
    if submit_case_response is None or submit_case_response.status_code != 201:

        status_code = submit_case_response.status_code if submit_case_response else "N/A"
        text = submit_case_response.text if submit_case_response else "No response from API"

        print(f"Case submission failed: {status_code} - {text}")

        result = {
            "caseNo": caseNo,
            "state": "Add state from event body",
            "status": "ERROR",
            "error": f"Case submission failed: {status_code} - {text}",
            "end_date_time": datetime.now(timezone.utc).isoformat()
        }
        return result

    else:
        result = {
            "runID": "get run id from event body",
            "caseNo": caseNo,
            "state": "Add state from event body",
            "status": "Success",
            "error": None,
            "end_date_time": datetime.now(timezone.utc).isoformat(),
            "ccd_case_id": submit_case_response.json()["id"]
        }
        return result










## Creating a dummy payload

This section is to test the functions and classes above to process a dummy payload

In [0]:
payload_data = """
{
  "email": "example@test.com",
  "isEjp": "No",
  "feeCode": "FEE0238",
  "isAdmin": "Yes",
  "paidDate": "2024-08-05",
  "appealType": "refusalOfHumanRights",
  "feeVersion": "2",
  "paidAmount": "14000",
  "s94bStatus": "No",
  "paymentDate": "5 Aug 2024",
  "feeAmountGbp": "14000",
  "isIntegrated": "No",
  "appellantInUk": "Yes",
  "hearingCentre": "taylorHouse",
  "isNabaEnabled": "No",
  "paymentStatus": "Paid",
  "staffLocation": "Taylor House",
  "SearchCriteria": {
    "SearchParties": [
      {
        "id": "ec889f66-0475-4633-8d69-b31b80d76e5a",
        "value": {
          "Name": "GivenName Migration 3 FamilyName appealSubmitted",
          "PostCode": "SE10 0XX",
          "DateOfBirth": "2000-01-01",
          "AddressLine1": "Flat 101",
          "EmailAddress": "example@test.com"
        }
      }
    ],
    "OtherCaseReferences": [
      {
        "id": "65e7cf55-21c9-4d5b-af62-afd13222a8eb",
        "value": "HU/50009/2024"
      }
    ]
  },
  "feeDescription": "Appeal determined with a hearing",
  "feeWithHearing": "140",
  "searchPostcode": "SE10 0XX",
  "hasOtherAppeals": "No",
  "adminDeclaration1": [
    "hasDeclared"
  ],
  "appellantAddress": {
    "County": "",
    "Country": "United Kingdom",
    "PostCode": "SE10 0XX",
    "PostTown": "London",
    "AddressLine1": "Flat 101",
    "AddressLine2": "10 Cutter Lane",
    "AddressLine3": ""
  },
  "appellantPartyId": "45889c92-2cf4-4dae-ae9a-f64aa051d525",
  "ariaDesiredState": "appealSubmitted",
  "isAppellantMinor": "No",
  "isNabaAdaEnabled": "No",
  "isNabaEnabledOoc": "No",
  "hearingTypeResult": "No",
  "hmctsCaseCategory": "Human rights",
  "notificationsSent": [],
  "tribunalDocuments": [],
  "appealOutOfCountry": "No",
  "appellantStateless": "hasNationality",
  "legalRepFamilyName": "",
  "paymentDescription": "Appeal determined with a hearing",
  "appellantFamilyName": "FamilyName appealSubmitted",
  "appellantGivenNames": "GivenName Migration 3",
  "isFeePaymentEnabled": "Yes",
  "isRemissionsEnabled": "Yes",
  "submissionOutOfTime": "No",
  "appealSubmissionDate": "2024-08-07",
  "appellantDateOfBirth": "2000-01-01",
  "feePaymentAppealType": "Yes",
  "letterSentOrReceived": "Sent",
  "localAuthorityPolicy": {
    "Organisation": {},
    "OrgPolicyCaseAssignedRole": "[LEGALREPRESENTATIVE]"
  },
  "tribunalReceivedDate": "2024-08-05",
  "additionalPaymentInfo": "Additional paid information",
  "appealReferenceNumber": "HU/50009/2024",
  "caseNameHmctsInternal": "GivenName Migration 3 FamilyName appealSubmitted",
  "hmctsCaseNameInternal": "GivenName Migration 3 FamilyName appealSubmitted",
  "isOutOfCountryEnabled": "Yes",
  "appellantNationalities": [
    {
      "id": "520cd556-39b3-4729-9093-a07513f4b03e",
      "value": {
        "code": "GB"
      }
    }
  ],
  "caseManagementCategory": {
    "value": {
      "code": "refusalOfHumanRights",
      "label": "Refusal of a human rights claim"
    },
    "list_items": [
      {
        "code": "refusalOfHumanRights",
        "label": "Refusal of a human rights claim"
      }
    ]
  },
  "caseManagementLocation": {
    "region": "1",
    "baseLocation": "765324"
  },
  "homeOfficeDecisionDate": "2024-08-05",
  "internalAppellantEmail": "example@test.com",
  "appealGroundsForDisplay": [],
  "appellantsRepresentation": "Yes",
  "appellantNameForDisplay": "GivenName Migration 3 FamilyName appealSubmitted",
  "deportationOrderOptions": "No",
  "uploadTheAppealFormDocs": [],
  "appellantHasFixedAddress": "Yes",
  "decisionHearingFeeOption": "decisionWithHearing",
  "hasServiceRequestAlready": "No",
  "homeOfficeReferenceNumber": "012345678",
  "isDlrmFeeRemissionEnabled": "Yes",
  "legalRepIndividualPartyId": "f7159136-7bff-40fb-921a-c8a53633afc8",
  "legalRepOrganisationPartyId": "71c50709-b802-42c7-ac56-2ef03e6e14e7",
  "appealSubmissionInternalDate": "2024-08-07",
  "ccdReferenceNumberForDisplay": "1723 0197 9804 1350",
  "legalRepresentativeDocuments": [],
  "sendDirectionActionAvailable": "Yes",
  "uploadTheNoticeOfDecisionDocs": [],
  "automaticEndAppealTimedEventId": "fd614594-6b6b-4116-8568-f0d80298486e",
  "currentCaseStateVisibleToJudge": "appealSubmitted",
  "currentCaseStateVisibleToCaseOfficer": "appealSubmitted",
  "changeDirectionDueDateActionAvailable": "No",
  "currentCaseStateVisibleToAdminOfficer": "appealSubmitted",
  "markEvidenceAsReviewedActionAvailable": "No",
  "uploadAddendumEvidenceActionAvailable": "No",
  "currentCaseStateVisibleToHomeOfficeAll": "appealSubmitted",
  "currentCaseStateVisibleToHomeOfficeApc": "appealSubmitted",
  "currentCaseStateVisibleToHomeOfficePou": "appealSubmitted",
  "currentCaseStateVisibleToHomeOfficeLart": "appealSubmitted",
  "uploadAdditionalEvidenceActionAvailable": "No",
  "applicationChangeDesignatedHearingCentre": "taylorHouse",
  "currentCaseStateVisibleToHomeOfficeGeneric": "appealSubmitted",
  "haveHearingAttendeesAndDurationBeenRecorded": "No",
  "currentCaseStateVisibleToLegalRepresentative": "appealSubmitted",
  "markAddendumEvidenceAsReviewedActionAvailable": "No",
  "uploadAddendumEvidenceLegalRepActionAvailable": "No",
  "isServiceRequestTabVisibleConsideringRemissions": "Yes",
  "uploadAddendumEvidenceHomeOfficeActionAvailable": "No",
  "uploadAddendumEvidenceAdminOfficerActionAvailable": "No",
  "uploadAdditionalEvidenceHomeOfficeActionAvailable": "No",
  "remissionType": "hoWaiverRemission",
  "ariaMigrationTaskDueDays": "2"
} """

In [0]:
import json
caseNo = "ABC/123"

# payload_data = json.dumps({
#   "data": payload_data,
#   "event": {"id":"ariaCreateCase"},
#   "event_token": event_token, 
#   "ignore_warning": True
# })

results = process_case(env="sbox",caseNo=caseNo,payload_data=payload_data,idam_token=idam_token,uid=uid,s2s_token=s2s_token)

In [0]:
print(results)